In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    default_data_collator,
    SchedulerType
)
import os
import json
from itertools import chain
from datasets import load_dataset

In [2]:
block_size = 32768
train_file = 'combine-v2.jsonl'
tokenizer = AutoTokenizer.from_pretrained(
    'mesolitica/llama-7b-hf-16384-fpf',
)
text_column_name = 'text'

In [4]:
!rm tmp*

rm: cannot remove 'tmp*': No such file or directory


In [5]:
raw_datasets = load_dataset(
    'json',
    data_files=train_file,
    split='train'
)

filename = os.path.split(train_file)[1]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

column_names = raw_datasets.column_names
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-tokenized-1024',
    num_proc=20,
)

In [6]:
def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-grouped-{block_size}',
    num_proc=20,
)

Map (num_proc=20):   0%|          | 0/36885044 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
len(lm_datasets[1]['input_ids'])

32768

In [9]:
tokenizer.decode(lm_datasets[1]['input_ids'])

'دڠDefinisi : 1. menyepak, mendepak, mene-rajang: ditendangnya pintu rumah itu dgn sekuat-kuat hatinya; 2. bp mengusir (dr per-satuan dll), memecat (dr jawatan dll): mereka ditendang keluar kerana melanggar disiplin parti; 3. mendesak kuat-kuat, menolak-nolak: buah dadanya yg tegang ~ colinya; (Kamus Dewan Edisi Keempat) [me.nen.dang] | منندڠDefinisi : 1 memukul dgn kaki; menyepak: Dia ~ bola itu ke pintu gol lawannya. 2 ki menyuruh berhenti drpd menjadi anggota sesuatu persatuan dll; memberhentikan drpd jawatan dll; memecat: Ahli jawatankuasa telah ~ beliau dr persatuan itu. (Kamus Pelajar Edisi Kedua) menendang tendangan</s><s> [me.me.sa.ni] | ممسنيDefinisi : berpesan (memesan) kpd: maka kata orang yg dipesani itu, “Baiklah”; (Kamus Dewan Edisi Keempat)</s><s> Definisi : tandanya: jika ada negara yg mengatur rancangan perangnya sendiri, ~ pakatan itu tidak akan mencapai tujuan bersama; (Kamus Dewan Edisi Keempat)</s><s> [gé.no.tip] | ݢينوتيڤDefinisi : (génotip) (Bio) 1. komposisi cir